# 闭包

简单的闭包示例

In [ ]:
def outer_function(x):
  def inner_function(y):
      return x + y
  return inner_function

closure = outer_function(10)
print(closure(5))    # 15
print(closure(10))   # 20

闭包的应用：保存状态

In [ ]:
def counter():
    count = 0
    def inner():
        nonlocal count
        count += 1
        return count
    return inner

c = counter()
print(c())    # 1
print(c())    # 2

# 工厂函数

案例1：日志封装

> 注意，以下代码没有输出，但是会创建两个文件：error.log 和 event.log

In [ ]:
def create_logger(log_file):
    def logger(message):
        with open(log_file, 'a') as file:
            file.write(message + '\n')
    return logger

error_logger = create_logger('error.log')
event_logger = create_logger('event.log')

error_logger('Error occurred: Division by zero')
event_logger('User "admin" logged in')

案例2：面积计算器

In [ ]:
class Circle:
    def __init__(self, radius):
        self.radius = radius
        
    def area(self):
        return 3.14 * self.radius ** 2
    
class Rectangle:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        
    def area(self):
        return self.width * self.height
    
def create_shape(shape_type, *args, **kwargs):
    if shape_type == 'circle':
        return Circle(*args, **kwargs)
    elif shape_type == 'rectangle':
        return Rectangle(*args, **kwargs)
    else:
        raise ValueError('Invalid shape type')
    
circle = create_shape('circle', 5)
print(circle.area())
rectangle = create_shape('rectangle', 3, 4)
print(rectangle.area())

# 装饰器

代码运行时间

In [ ]:
import time

def benchmark(func):
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        return_value = func(*args, **kwargs)
        end = time.perf_counter()
        print(f'函数 {func.__name__} 运行耗时 {end - start:.2f} 秒')
        return return_value
    return wrapper

@benchmark
def count_prime_number(n):
    count = 0
    for num in range(2, n + 1):
        prime = True
        for i in range(2, num):
            if num % i == 0:
                prime = False
                break
        if prime:
            count += 1
    return count

count_prime_number(10000)

命令解析

In [ ]:
def cmd_register():
    commands = {}
    def register(name):
        def wrapper(func):
            commands[name] = func
            return func
        return wrapper
    def run(name, *args, **kwargs):
        return commands[name](*args, **kwargs)
    return register, run

command, runner = cmd_register()

@command('add')
def add(a, b):
    return a + b

@command('sub')
def sub(a, b):
    return a - b

print(runner('add', 1, 2))
print(runner('sub', 3, 1))

并不实际的实际应用

但理解它是真正的挑战

In [ ]:
from numbers import Number

def cmd_register():
    commands = {}

    def register(name):
        def wrapper(func):
            commands[name] = func
            return func
        return wrapper

    def run(name, *args, **kwargs):
        return commands[name](*args, **kwargs)
    return register, run

def checker(args_checker, kwargs_checker):
    # 传入的两个检查器是两个函数，分别检查args和kwargs；如果是Bool类型-> True表示始终通过检查，False表示不能有参数
    def wrapper(func):
        def inner(*args, **kwargs):
            for val, check in zip([args, kwargs], [args_checker, kwargs_checker]):
                if check is True:
                    continue
                elif check is False and len(val) > 0:
                    raise ValueError(f"Argument error in function {func.__name__}")
                elif not isinstance(check, bool) and not check(val):
                    raise ValueError(f"Argument check failed in function {func.__name__}")
            return func(*args, **kwargs)
        return inner
    return wrapper

command, runner = cmd_register()

@command('add')
# 这里的匿名函数检查条件：传入参数都是Number类型
@checker(lambda x: all(isinstance(i, Number) for i in x), False)
def add(*args):
    return sum(args)

@command('dev')
# 这里的匿名函数检查条件：传入参数都是Number类型，且只有两个参数，且第二个参数不能为0
@checker(lambda x: all(isinstance(i, Number) for i in x) and len(x) == 2 and x[1]!=0, False)
def dev(*args):
    return args[0] / args[1]

if __name__ == '__main__':
    print(runner('add', 1, 2))
    print(runner('dev', 3, 1))